In [60]:
# Libraries needed (pandas is not standard and must be installed in Python)
import requests
import pandas as pd
from datetime import datetime, timedelta

# Insert your own client ID here
client_id = '0a280373-5d8b-450a-8374-02491cfbcaf8'

In [61]:
endpoint = 'https://frost.met.no/observations/v0.jsonld'
parameters = {
    'sources': 'SN18700',
    'elements': 'max(air_temperature PT1H), sum(precipitation_amount PT1H), max(wind_speed PT1H), boolean_clear_sky_weather(cloud_area_fraction P1D)',
    'referencetime': '2022-11-01/2025-11-01',
    'timeoffsets': 'default',
    'levels': 'default'
}

r = requests.get(endpoint, parameters, auth=(client_id, ''))
data = r.json()

In [62]:
# Check if the request worked, print out any errors
if r.status_code == 200:
    data_json = r.json()     # store the json safely
    print("Data retrieved from frost.met.no!")

else:
    data_json = r.json()
    print("Error! Returned status code %s" % r.status_code)
    print("Message: %s" % data_json['error']['message'])
    print("Reason: %s" % data_json['error']['reason'])


Data retrieved from frost.met.no!


In [63]:
rows = []  # collect all rows here

for item in data_json["data"]:
    if "observations" not in item:
        continue  # skip if no observations

    # Convert list of observations to a temporary DataFrame
    row = pd.DataFrame(item["observations"])

    # Add shared metadata
    row["referenceTime"] = item.get("referenceTime")
    row["sourceId"] = item.get("sourceId")

    rows.append(row)

# Concatenate everything at once (fast!)
df = pd.concat(rows, ignore_index=True)


In [64]:
#df = df[df["timeResolution"] == "PT1H"]

In [65]:
df.head()  # show the first few rows

,elementId,value,unit,level,timeOffset,timeResolution,timeSeriesId,performanceCategory,exposureCategory,qualityCode,referenceTime,sourceId
0,max(air_temperature PT1H),8.7,degC,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,PT1H,0,C,1,0,2022-11-01T00:00:00.000Z,SN18700:0
1,sum(precipitation_amount PT1H),0.1,mm,NaN,PT0H,PT1H,0,C,2,0,2022-11-01T00:00:00.000Z,SN18700:0
2,max(wind_speed PT1H),1.5,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,PT1H,0,C,2,0,2022-11-01T00:00:00.000Z,SN18700:0
3,boolean_clear_sky_weather(cloud_area_fraction ...,0.0,sum,NaN,PT0H,P1D,0,C,2,2,2022-11-01T00:00:00.000Z,SN18700:0
4,max(air_temperature PT1H),8.5,degC,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,PT1H,0,C,1,0,2022-11-01T01:00:00.000Z,SN18700:0


In [66]:
wanted_elements = [
    "max(air_temperature PT1H)",
    "sum(precipitation_amount PT1H)",
    "max(wind_speed PT1H)"
]
df_filtered = df[df["elementId"].isin(wanted_elements)]
df_filtered.head()


df

,elementId,value,unit,level,timeOffset,timeResolution,timeSeriesId,performanceCategory,exposureCategory,qualityCode,referenceTime,sourceId
0,max(air_temperature PT1H),8.7,degC,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,PT1H,0,C,1,0,2022-11-01T00:00:00.000Z,SN18700:0
1,sum(precipitation_amount PT1H),0.1,mm,NaN,PT0H,PT1H,0,C,2,0,2022-11-01T00:00:00.000Z,SN18700:0
2,max(wind_speed PT1H),1.5,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,PT1H,0,C,2,0,2022-11-01T00:00:00.000Z,SN18700:0
3,boolean_clear_sky_weather(cloud_area_fraction ...,0.0,sum,NaN,PT0H,P1D,0,C,2,2,2022-11-01T00:00:00.000Z,SN18700:0
4,max(air_temperature PT1H),8.5,degC,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,PT1H,0,C,1,0,2022-11-01T01:00:00.000Z,SN18700:0
...,...,...,...,...,...,...,...,...,...,...,...,...
79832,sum(precipitation_amount PT1H),0.0,mm,NaN,PT0H,PT1H,0,C,2,0,2025-10-31T22:00:00.000Z,SN18700:0
79833,max(wind_speed PT1H),2.6,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,PT1H,0,C,2,0,2025-10-31T22:00:00.000Z,SN18700:0
79834,max(air_temperature PT1H),2.5,degC,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,PT1H,0,C,1,0,2025-10-31T23:00:00.000Z,SN18700:0
79835,sum(precipitation_amount PT1H),0.0,mm,NaN,PT0H,PT1H,0,C,2,0,2025-10-31T23:00:00.000Z,SN18700:0


In [67]:
df_pivot = df_filtered.pivot_table(
    index="referenceTime",   # your datetime column
    columns="elementId",     # elementId becomes column headers
    values="value",          # the measurement becomes cell values
    aggfunc="first"          # avoids errors if duplicates exist
).reset_index()

df_pivot.rename(columns={"referenceTime" : "Datetime(utc)", "max(air_temperature PT1H)" : "temprature(celsius)", "max(wind_speed PT1H)" : "wind_speed(m/s)", "sum(precipitation_amount PT1H)" : "precipitation(mm)"}, inplace=True)

df_pivot['Datetime(utc)'] = pd.to_datetime(df_pivot['Datetime(utc)'])
df_pivot["Date"] = df_pivot['Datetime(utc)'].dt.date

# Ensure types (but these lines do nothing unless you reassign)
df_pivot["precipitation(mm)"] = df_pivot["precipitation(mm)"].astype(float)
df_pivot["temprature(celsius)"] = df_pivot["temprature(celsius)"].astype(float)

def weather_event(row):
    if row["precipitation(mm)"] > 0:
        if row["temprature(celsius)"] <= 1.0:
            return "Snowy"
        else:
            return "Rainy"
    else:
        return "Dry"

df_pivot["Weather_Event"] = df_pivot.apply(weather_event, axis=1)



df_pivot.head()

elementId,Datetime(utc),temprature(celsius),wind_speed(m/s),precipitation(mm),Date,Weather_Event
0,2022-11-01 00:00:00+00:00,8.7,1.5,0.1,2022-11-01,Rainy
1,2022-11-01 01:00:00+00:00,8.5,1.4,0.0,2022-11-01,Dry
2,2022-11-01 02:00:00+00:00,8.5,1.3,0.0,2022-11-01,Dry
3,2022-11-01 03:00:00+00:00,8.5,1.5,0.0,2022-11-01,Dry
4,2022-11-01 04:00:00+00:00,8.4,0.9,0.0,2022-11-01,Dry


In [68]:
df_pivot.Weather_Event.value_counts()

Weather_Event
Dry      22629
Rainy     2743
Snowy      932
Name: count, dtype: int64

In [69]:
clouds = df[df["elementId"] == "boolean_clear_sky_weather(cloud_area_fraction P1D)"]
#clouds.head()

#clouds.timeResolution.value_counts(ascending=False)

In [70]:
#df.info()

In [77]:
df_clouds = clouds[["elementId", "referenceTime", "value"]]

df_clouds["referenceTime"] = pd.to_datetime(df_clouds["referenceTime"]).dt.date

#make value a int
df_clouds["value"] = df_clouds["value"].astype(int)

def weather_cloudy(row):
    if row["value"] == 0:
        return "Cloudy"
    else:
        return "Clear"

df_clouds["Cloudcover"] = df_clouds.apply(weather_cloudy, axis=1)

df_clouds.head()

/var/folders/w7/nmtncxts1kl0347wfqk9_kmr0000gn/T/ipykernel_75390/3146478340.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clouds["referenceTime"] = pd.to_datetime(df_clouds["referenceTime"]).dt.date
/var/folders/w7/nmtncxts1kl0347wfqk9_kmr0000gn/T/ipykernel_75390/3146478340.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clouds["value"] = df_clouds["value"].astype(int)
/var/folders/w7/nmtncxts1kl0347wfqk9_kmr0000gn/T/ipykernel_75390/3146478340.py:14: SettingWithCopyWarning: 
A value is tryin

,elementId,referenceTime,value,Cloudcover
3,boolean_clear_sky_weather(cloud_area_fraction ...,2022-11-01,0,Cloudy
76,boolean_clear_sky_weather(cloud_area_fraction ...,2022-11-02,0,Cloudy
149,boolean_clear_sky_weather(cloud_area_fraction ...,2022-11-03,0,Cloudy
222,boolean_clear_sky_weather(cloud_area_fraction ...,2022-11-04,0,Cloudy
295,boolean_clear_sky_weather(cloud_area_fraction ...,2022-11-05,0,Cloudy


In [80]:
df_merged = df_pivot.merge(df_clouds[["referenceTime", "value", "Cloudcover"]], left_on="Date", right_on="referenceTime", how="left")

df_merged.rename(columns={"value" : "cloud_area_fraction"}, inplace=True)
df_merged.drop(columns=["referenceTime"], inplace=True)
df_merged.drop(columns=["cloud_area_fraction"], inplace=True)
#remove index
df_merged.reset_index(drop=True, inplace=True)



df_merged.head()

,Datetime(utc),temprature(celsius),wind_speed(m/s),precipitation(mm),Date,Weather_Event,Cloudcover
0,2022-11-01 00:00:00+00:00,8.7,1.5,0.1,2022-11-01,Rainy,Cloudy
1,2022-11-01 01:00:00+00:00,8.5,1.4,0.0,2022-11-01,Dry,Cloudy
2,2022-11-01 02:00:00+00:00,8.5,1.3,0.0,2022-11-01,Dry,Cloudy
3,2022-11-01 03:00:00+00:00,8.5,1.5,0.0,2022-11-01,Dry,Cloudy
4,2022-11-01 04:00:00+00:00,8.4,0.9,0.0,2022-11-01,Dry,Cloudy


In [73]:
#Save to CSV
df_merged.to_csv("weather_data_oslo_2022_2025.csv", index=False)



